**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

    100% |████████████████████████████████| 2.3MB 11.4MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act funtion is the one that aloows us to choose what to do. It's that one that allows us to choose between the exploitation (using the learned best action) and the exploration (trying a new action to explore new possibilities and discoverf better behaviors). This exploration is controled  with the epsilon factor.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=3 # set small when debugging
epochs_test=3 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array is useful because it allows us to choose the eligible positions. Since the player have a vision of 2 squares around him, the unavailable positions have a -1 value and the player has a value of 1. The rest have a value of zero.


The borad game give the player the information about the surrounding bonus and malus on the map around him.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epochs):

        ##### FILL IN HERE

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    
        win = 0
        lose = 0
        game_over = False
        
        while not game_over :

            # The agent performs an action
            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/18.0. Average score (-6.5)
Win/lose count 5.5/14.0. Average score (-7.5)
Win/lose count 15.5/12.0. Average score (-3.8333333333333335)
Final score: -3.8333333333333335


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have first:

$$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{0\leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$$


$$Q^\pi(s,a)=E_{p^{\pi}}[r(s_0, a_0) \gamma^0 +  \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$$


$$Q^\pi(s,a)=E_{p^{\pi}}[r(s, a)  +  \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$$

$$Q^\pi(s,a)=E_{p^{\pi}}[r(s, a)  +  \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$$


$$Q^\pi(s,a)=E_{p^{\pi}}[r(s, a)  + \gamma \sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$$



We have for all p:

$$Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

so it's also the case for $\pi^*$:

$$Q^*(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^*(s',a')]$$

Since we have by definitnion:

$$Q^*(s,\pi^*(s))=\max_{a}Q^(s,a)$$

thus the final result:

$$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$



Now let's look at the loss function:

We could want to have as a target the $r +\gamma \max_aQ(s,a, \theta)$  which would be a nice target for $Q(a,s,\theta)$. we know that the difference si null wthen $\pi = \pi^*$, which means that reducing the difference which is the temporal difference is a nice objective to converge to $Q^*$. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory = self.memory[-self.max_memory:] + [m]
        pass

    def random_access(self):
        i = np.random.choice(min(len(self.memory), self.max_memory))
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
                        
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            state_, next_state_, action_, reward_, game_over_ = self.memory.random_access()
            input_states[i] = state_
            target_q[i] = self.model.predict(np.expand_dims(state_, 0))
            
            if game_over_:
                ######## FILL IN
                target_q[i, action_] = reward_
            else:
                ######## FILL IN
                target_q[i, action_] = reward_ + self.discount *\
                                       np.max(self.model.predict(np.expand_dims(next_state_, 0)))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2, 2)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model

        # This returns a tensor
        inputs = keras.layers.Input(shape=(5,5,self.n_state))
        flat = keras.layers.Flatten()(inputs)

        # a layer instance is callable on a tensor, and returns a tensor
        x = keras.layers.Dense(32, activation='relu')(flat)
        x = keras.layers.Dense(32, activation='relu')(x)
        predictions = keras.layers.Dense(4)(x)

        # This creates a model that includes
        # the Input layer and three Dense layers
        model = keras.models.Model(inputs=inputs, outputs=predictions)
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
        

In [34]:
epochs_train=30 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
for lr in [ 0.01]:
    print(lr)
    agent = DQN_FC(size, lr=lr, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

0.01
Epoch 000/030 | Loss 0.0123 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/030 | Loss 0.0433 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/030 | Loss 0.0707 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/030 | Loss 0.0301 | Win/lose count 0.5/3.0 (-2.5)
Epoch 004/030 | Loss 0.0355 | Win/lose count 2.5/1.0 (1.5)
Epoch 005/030 | Loss 0.0318 | Win/lose count 1.0/5.0 (-4.0)
Epoch 006/030 | Loss 0.0550 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/030 | Loss 0.0167 | Win/lose count 2.5/2.0 (0.5)
Epoch 008/030 | Loss 0.0159 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/030 | Loss 0.0157 | Win/lose count 1.5/4.0 (-2.5)
Epoch 010/030 | Loss 0.0216 | Win/lose count 2.0/2.0 (0.0)
Epoch 011/030 | Loss 0.0352 | Win/lose count 0.5/4.0 (-3.5)
Epoch 012/030 | Loss 0.0117 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/030 | Loss 0.0074 | Win/lose count 2.0/0 (2.0)
Epoch 014/030 | Loss 0.0104 | Win/lose count 1.0/1.0 (0.0)
Epoch 015/030 | Loss 0.0070 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/030 | Loss 0.0138 | Win/lose count 1.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        inputs = keras.layers.Input(shape=(5,5,self.n_state))

        x = Conv2D(16, (3, 3), padding='valid', activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Conv2D(16, (3, 3), padding='valid', activation='relu')(x)
        x = BatchNormalization()(x)

        x = keras.layers.Flatten()(x)

        predictions = keras.layers.Dense(4)(x)
        
        # This creates a model that includes
        # the Input layer and three Dense layers
        model = keras.models.Model(inputs=inputs, outputs=predictions)

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
for lr in [ 0.01]:
    print(lr)
    agent = DQN_CNN(size, lr=lr, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

0.01
Epoch 000/030 | Loss 0.2139 | Win/lose count 1.0/3.0 (-2.0)
Epoch 001/030 | Loss 0.0887 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/030 | Loss 0.1413 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/030 | Loss 0.0598 | Win/lose count 0.5/1.0 (-0.5)
Epoch 004/030 | Loss 0.0368 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/030 | Loss 0.0715 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/030 | Loss 0.0304 | Win/lose count 1.0/2.0 (-1.0)
Epoch 007/030 | Loss 0.1502 | Win/lose count 1.5/4.0 (-2.5)
Epoch 008/030 | Loss 0.0428 | Win/lose count 2.0/4.0 (-2.0)
Epoch 009/030 | Loss 0.0620 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/030 | Loss 0.0135 | Win/lose count 2.0/0 (2.0)
Epoch 011/030 | Loss 0.0251 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/030 | Loss 0.0214 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/030 | Loss 0.0267 | Win/lose count 3.5/3.0 (0.5)
Epoch 014/030 | Loss 0.0237 | Win/lose count 1.5/1.0 (0.5)
Epoch 015/030 | Loss 0.0422 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/030 | Loss 0.0223 | Win/lose count 1.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
epochs_test=30 # set small when debugging

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=lr, epsilon = 0.03, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=lr, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/3.0. Average score (-1.0)
Win/lose count 0.5/3.0. Average score (-1.5)
Win/lose count 0/0. Average score (-1.125)
Win/lose count 1.0/1.0. Average score (-0.9)
Win/lose count 0.5/1.0. Average score (-0.8333333333333334)
Win/lose count 0.5/0. Average score (-0.6428571428571429)
Win/lose count 1.0/1.0. Average score (-0.5625)
Win/lose count 1.5/0. Average score (-0.3333333333333333)
Win/lose count 1.0/0. Average score (-0.2)
Win/lose count 0.5/0. Average score (-0.13636363636363635)
Win/lose count 0.5/0. Average score (-0.08333333333333333)
Win/lose count 0/2.0. Average score (-0.23076923076923078)
Win/lose count 0.5/0. Average score (-0.17857142857142858)
Win/lose count 0/1.0. Average score (-0.23333333333333334)
Win/lose count 0.5/0. Average score (-0.1875)
Win/lose count 1.0/1.0. Average score (-0.17647058823529413)
Win/lose count 1.0/1.0. Average score (-0.16666666666666666)
Win/lose count 0/1.0. Average scor

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

Given the structure of the data CNN are supposed to outperform fully connected network, which it does but not consistently.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,stop_epsilon,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        agent.set_epsilon((1 -min(1, e/stop_epsilon))*agent.epsilon)
        
        print(min(1, e/stop_epsilon), (1 -min(1, e/stop_epsilon))*agent.epsilon)

        win = 0
        lose = 0

        while not game_over:            
            action = agent.act(state)
            
            prev_state = state
            state, reward, game_over = env.act(action, train=True) 

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring,self).__init__(grid_size, max_time, temperature)
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        
    def act(self, action, train=False):
        
        state, reward, game_over = super(EnvironmentExploring,self).act(action)

        if train:
            reward += -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        # take malus_position into account
        state = self.get_state()

        return state, reward, game_over
    
    def get_state(self):
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    def reset(self):
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        # the rest is the same as for the parent class
        super(EnvironmentExploring, self).reset()   
        
        return self.get_state()




In [0]:
# Training
stop_epsilon = 45
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train,stop_epsilon, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

0.0 0.1
Epoch 000/030 | Loss 0.0131 | Win/lose count 3.0/21.80000000000005 (-18.80000000000005)
0.022222222222222223 0.09560493827160493
Epoch 001/030 | Loss 0.0078 | Win/lose count 2.0/20.900000000000013 (-18.900000000000013)
0.044444444444444446 0.08927956104252402
Epoch 002/030 | Loss 0.0069 | Win/lose count 2.0/22.000000000000018 (-20.000000000000018)
0.06666666666666667 0.08138973936899864
Epoch 003/030 | Loss 0.0038 | Win/lose count 1.0/21.200000000000035 (-20.200000000000035)
0.08888888888888889 0.07238949834882895
Epoch 004/030 | Loss 0.0015 | Win/lose count 2.0/21.000000000000018 (-19.000000000000018)
0.1111111111111111 0.06277680073611182
Epoch 005/030 | Loss 0.0031 | Win/lose count 0.5/20.10000000000002 (-19.60000000000002)
0.13333333333333333 0.05304639662201449
Epoch 006/030 | Loss 0.0059 | Win/lose count 1.5/23.80000000000004 (-22.30000000000004)
0.15555555555555556 0.04364615197845523
Epoch 007/030 | Loss 0.0016 | Win/lose count 1.5/21.100000000000037 (-19.60000000000003

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))in a 

Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)


NameError: ignored

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***